In [ ]:
# op1
# data = data.assign(datetime = pd.to_datetime(data['Date'], format="%d/%m/%Y"))

# op2
# data["datetime"] = pd.to_datetime(data['Date'], format="%d/%m/%Y")

In [ ]:
# data = data.set_index('datetime').drop(['Day', 'Date'], axis=1)

In [ ]:
# # pandas quick plotting method
# data['totalU'].plot()